In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [2]:

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

##  Исследуйте таблицы — выведите первые строки

In [3]:
query = ''' SELECT *
            FROM books
            LIMIT 5
        '''

pd.io.sql.read_sql(query, con = engine)         

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query = ''' SELECT *
            FROM authors
            LIMIT 5
        '''

pd.io.sql.read_sql(query, con = engine)     

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
query = ''' SELECT *
            FROM publishers

            LIMIT 5
        '''

pd.io.sql.read_sql(query, con = engine) 

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
query = ''' SELECT *
            FROM ratings

            LIMIT 5
        '''

pd.io.sql.read_sql(query, con = engine) 

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
query = ''' SELECT *
            FROM reviews

            LIMIT 5
        '''

pd.io.sql.read_sql(query, con = engine) 

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


# Посчитайте, сколько книг вышло после 1 января 2000 года;

In [8]:
query = ''' SELECT COUNT(book_id)
            FROM books

            WHERE CAST(publication_date AS date) > '2000-01-01'

        '''

pd.io.sql.read_sql(query, con = engine) 

,count
0,819


После первого января 2000 года вышло 819 книг

# Для каждой книги посчитайте количество обзоров и среднюю оценку

In [9]:
query = ''' SELECT AVG(r.rating),
                   COUNT(re.review_id),
                   b.title
            FROM ratings AS r JOIN reviews AS re ON r.book_id = re.book_id
            JOIN books AS b ON re.book_id = b.book_id
            GROUP BY b.title
            ORDER BY  COUNT(re.review_id) DESC;

            

        '''

pd.io.sql.read_sql(query, con = engine) 

,avg,count,title
0,3.662500,1120,Twilight (Twilight #1)
1,4.125000,528,The Hobbit or There and Back Again
2,3.825581,516,The Catcher in the Rye
3,4.414634,492,Harry Potter and the Prisoner of Azkaban (Harr...
4,4.287500,480,Harry Potter and the Chamber of Secrets (Harry...
...,...,...,...
988,5.000000,2,The Adventures of Tom Sawyer and Adventures of...
989,2.500000,2,History of Beauty
990,4.000000,2,Lysistrata
991,3.500000,2,Executive Orders (Jack Ryan #8)


In [10]:
query = ''' with
            r as (SELECT AVG(rating) AS avg_rating,
                   book_id
                   
            FROM ratings
            GROUP BY book_id),
            
            re as (SELECT COUNT(review_id) AS rev_count,
                          book_id
                   
                   FROM reviews 
                   GROUP BY book_id),
                   
            
            d as (select re.rev_count as rr,
                   r.avg_rating,
                   b.title
            from r LEFT JOIN re ON r.book_id=re.book_id
            JOIN books AS b ON r.book_id=b.book_id)
            
            select sum(d.rr)
            from d
            

        '''

pd.io.sql.read_sql(query, con = engine) 

,sum
0,2793.0


In [11]:
query = ''' with
            r as (SELECT AVG(rating) AS avg_rating,
                   book_id
                   
            FROM ratings
            GROUP BY book_id),
            
            re as (SELECT COUNT(review_id) AS rev_count,
                          book_id
                   
                   FROM reviews 
                   GROUP BY book_id)
                   
            
            select re.rev_count,
                   r.avg_rating,
                   b.title
            from r LEFT JOIN re ON r.book_id=re.book_id
            JOIN books AS b ON r.book_id=b.book_id
            
            

        '''

pd.io.sql.read_sql(query, con = engine)

,rev_count,avg_rating,title
0,2.0,3.666667,'Salem's Lot
1,1.0,2.500000,1 000 Places to See Before You Die
2,3.0,4.666667,13 Little Blue Envelopes (Little Blue Envelope...
3,2.0,4.500000,1491: New Revelations of the Americas Before C...
4,4.0,4.000000,1776
...,...,...,...
995,3.0,3.666667,Wyrd Sisters (Discworld #6; Witches #2)
996,3.0,3.400000,Xenocide (Ender's Saga #3)
997,4.0,3.200000,Year of Wonders
998,2.0,4.500000,You Suck (A Love Story #2)


Наилучшие показатели по средней оценке и кол-ву обзоров у книг - A Dirty Job (Grim Reaper #1) (рейтинг 5, кол-во обзоров 16),
School's Out—Forever (Maximum Ride #2) (5 и 12) и Moneyball: The Art of Winning an Unfair Game (5 и 9)

Самыми обсуждаемыми являются: Twilight (Twilight #1)(1120 обзоров), The Hobbit or There and Back Again (528), The Catcher in the Rye (516), Harry Potter and the Prisoner of Azkaban (492) и Harry Potter and the Chamber of Secrets (480)

# Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [12]:
query = ''' SELECT p.publisher,
                   COUNT(b.book_id)
            FROM publishers AS p JOIN books AS b ON p.publisher_id = b.publisher_id
            WHERE b.num_pages > 50
            GROUP BY p.publisher
            order by COUNT(b.book_id) desc
            LIMIT 1


            

        '''

pd.io.sql.read_sql(query, con = engine) 

,publisher,count
0,Penguin Books,42


Издательство, которое выпустило наибольшее кол-во книг - Penguin Books

# Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [13]:
query = ''' WITH
            f AS (SELECT AVG(r.rating) AS avg_rating,
                        r.book_id AS book_id,
                        COUNT(r.rating_id) 
                  FROM ratings AS r
                  GROUP BY  r.book_id
                  HAVING COUNT(r.rating_id) >= 50
                  ORDER BY avg_rating DESC)
      
                SELECT a.author,
                       AVG(f.avg_rating)
                       
                FROM f LEFT JOIN books AS b ON f.book_id = b.book_id
                LEFT JOIN authors AS a ON b.author_id = a.author_id
                GROUP BY a.author
                ORDER BY AVG(f.avg_rating) DESC
                LIMIT 1
                


            

        '''

pd.io.sql.read_sql(query, con = engine)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844


Автор с наивысшей средней оценкой - J.K. Rowling/Mary GrandPré	

# Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [14]:
query = ''' WITH
            f AS (SELECT 
                        r.username AS users_50,
                        COUNT(r.rating_id) 
                  FROM ratings AS r
                  GROUP BY  users_50
                  HAVING COUNT(r.rating_id) > 50),
      
            sd AS (SELECT  count(re.review_id) as cnt_re,
                        f.users_50
                FROM f left JOIN reviews AS re ON f.users_50 = re.username
                group by f.users_50)  

                SELECT AVG(cnt_re)
                FROM sd
                
                


            

        '''

pd.io.sql.read_sql(query, con = engine)

,avg
0,24.333333


В среднем на пользователя, который поставил больше 50 оценок, приходится по 24 обзора

# Вывод

Если взять во внимание все полученные результаты, то компании стоит обратить внимание на автора с наибольшей средней оценкой - J.K. Rowling/Mary GrandPré, издательство с хорошими показателями выпуска книг - Penguin Books и на самые обсуждаемые и с наибольшей средней оценкой книги - A Dirty Job (Grim Reaper #1) (рейтинг 5, кол-во обзоров 16),
School's Out—Forever (Maximum Ride #2) (5 и 12) и Moneyball: The Art of Winning an Unfair Game (5 и 9). 

Еще на самые обсуждаемые, но с не самыми высокими средними оценками книги: Twilight (Twilight #1)(1120 обзоров), The Hobbit or There and Back Again (528), The Catcher in the Rye (516), Harry Potter and the Prisoner of Azkaban (492) и Harry Potter and the Chamber of Secrets (480)